In [76]:
import polars as pl
from kmodes.kmodes import KModes
from polars import DataFrame
from sklearn import preprocessing
import altair as alt

## Load the dataset

In [77]:
df = pl.read_csv('Data-Mushroom/mushrooms.csv', skip_rows=2)
df.head()

class,cap-shape,cap-surface,cap-color,bruises,odor,gill-attachment,gill-spacing,gill-size,gill-color,stalk-shape,stalk-root,stalk-surface-above-ring,stalk-surface-below-ring,stalk-color-above-ring,stalk-color-below-ring,veil-type,veil-color,ring-number,ring-type,spore-print-color,population,habitat
str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str
"""p""","""x""","""s""","""n""","""t""","""p""","""f""","""c""","""n""","""k""","""e""","""e""","""s""","""s""","""w""","""w""","""p""","""w""","""o""","""p""","""k""","""s""","""u"""
"""e""","""x""","""s""","""y""","""t""","""a""","""f""","""c""","""b""","""k""","""e""","""c""","""s""","""s""","""w""","""w""","""p""","""w""","""o""","""p""","""n""","""n""","""g"""
"""e""","""b""","""s""","""w""","""t""","""l""","""f""","""c""","""b""","""n""","""e""","""c""","""s""","""s""","""w""","""w""","""p""","""w""","""o""","""p""","""n""","""n""","""m"""
"""p""","""x""","""y""","""w""","""t""","""p""","""f""","""c""","""n""","""n""","""e""","""e""","""s""","""s""","""w""","""w""","""p""","""w""","""o""","""p""","""k""","""s""","""u"""
"""e""","""x""","""s""","""g""","""f""","""n""","""f""","""w""","""b""","""k""","""t""","""e""","""s""","""s""","""w""","""w""","""p""","""w""","""o""","""e""","""n""","""a""","""g"""


In [78]:
df.describe()

statistic,class,cap-shape,cap-surface,cap-color,bruises,odor,gill-attachment,gill-spacing,gill-size,gill-color,stalk-shape,stalk-root,stalk-surface-above-ring,stalk-surface-below-ring,stalk-color-above-ring,stalk-color-below-ring,veil-type,veil-color,ring-number,ring-type,spore-print-color,population,habitat
str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str
"""count""","""8124""","""8124""","""8124""","""8124""","""8124""","""8124""","""8124""","""8124""","""8124""","""8124""","""8124""","""8124""","""8124""","""8124""","""8124""","""8124""","""8124""","""8124""","""8124""","""8124""","""8124""","""8124""","""8124"""
"""null_count""","""0""","""0""","""0""","""0""","""0""","""0""","""0""","""0""","""0""","""0""","""0""","""0""","""0""","""0""","""0""","""0""","""0""","""0""","""0""","""0""","""0""","""0""","""0"""
"""mean""",null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
"""std""",null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
"""min""","""e""","""b""","""f""","""b""","""f""","""a""","""a""","""c""","""b""","""b""","""e""","""?""","""f""","""f""","""b""","""b""","""p""","""n""","""n""","""e""","""b""","""a""","""d"""
"""25%""",null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
"""50%""",null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
"""75%""",null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
"""max""","""p""","""x""","""y""","""y""","""t""","""y""","""f""","""w""","""n""","""y""","""t""","""r""","""y""","""y""","""y""","""y""","""p""","""y""","""t""","""p""","""y""","""y""","""w"""


In [79]:
y = df.select('class')
x = df.drop('class')

In [80]:
x.schema

Schema([('cap-shape', String),
        ('cap-surface', String),
        ('cap-color', String),
        ('bruises', String),
        ('odor', String),
        ('gill-attachment', String),
        ('gill-spacing', String),
        ('gill-size', String),
        ('gill-color', String),
        ('stalk-shape', String),
        ('stalk-root', String),
        ('stalk-surface-above-ring', String),
        ('stalk-surface-below-ring', String),
        ('stalk-color-above-ring', String),
        ('stalk-color-below-ring', String),
        ('veil-type', String),
        ('veil-color', String),
        ('ring-number', String),
        ('ring-type', String),
        ('spore-print-color', String),
        ('population', String),
        ('habitat', String)])

In [81]:
df.null_count()

class,cap-shape,cap-surface,cap-color,bruises,odor,gill-attachment,gill-spacing,gill-size,gill-color,stalk-shape,stalk-root,stalk-surface-above-ring,stalk-surface-below-ring,stalk-color-above-ring,stalk-color-below-ring,veil-type,veil-color,ring-number,ring-type,spore-print-color,population,habitat
u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


## Preprocessing data


In [82]:
encoder = preprocessing.LabelEncoder()
x = x.select(
    [
        pl.all().map_batches(encoder.fit_transform)
    ]
)

In [83]:
x.head()

cap-shape,cap-surface,cap-color,bruises,odor,gill-attachment,gill-spacing,gill-size,gill-color,stalk-shape,stalk-root,stalk-surface-above-ring,stalk-surface-below-ring,stalk-color-above-ring,stalk-color-below-ring,veil-type,veil-color,ring-number,ring-type,spore-print-color,population,habitat
i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64
5,2,4,1,6,1,0,1,4,0,3,2,2,7,7,0,2,1,4,2,3,5
5,2,9,1,0,1,0,0,4,0,2,2,2,7,7,0,2,1,4,3,2,1
0,2,8,1,3,1,0,0,5,0,2,2,2,7,7,0,2,1,4,3,2,3
5,3,8,1,6,1,0,1,5,0,3,2,2,7,7,0,2,1,4,2,3,5
5,2,3,0,5,1,1,0,4,1,3,2,2,7,7,0,2,1,0,3,0,1


## Clustering using K-Modes


In [84]:
km = KModes(n_clusters=2, init='Huang', n_init=5, verbose=1)  # Huang, Cao, random

In [85]:
clusters = km.fit_predict(x)

Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 1, iteration: 1/100, moves: 1727, cost: 63002.0
Run 1, iteration: 2/100, moves: 37, cost: 63002.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 2, iteration: 1/100, moves: 606, cost: 62730.0
Run 2, iteration: 2/100, moves: 168, cost: 62730.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 3, iteration: 1/100, moves: 759, cost: 63138.0
Run 3, iteration: 2/100, moves: 47, cost: 63138.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 4, iteration: 1/100, moves: 2135, cost: 62534.0
Run 4, iteration: 2/100, moves: 249, cost: 62534.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 5, iteration: 1/100, moves: 419, cost: 63615.0
Best run was number 4


In [86]:
yp = DataFrame({
    'predicted-label': clusters
})
yp.head()

predicted-label
u16
0
0
0
0
0


In [101]:
counting = yp.select(pl.col('predicted-label').value_counts()).unnest('predicted-label')

pie = (alt.Chart(counting).mark_arc()
.encode(
    theta=alt.Theta('count', stack='normalize', type='quantitative'),
    color=alt.Color('predicted-label', type='nominal'),
    tooltip=['predicted-label', 'count:N'],
))

pie

alt.Chart(...)